In [ ]:
# code


# WITH    CTE1  
#   AS ( 
#     select distinct crn,ACCOUNT_NO,cast(CREATED_DATE as Date) as ACCNT_OPN_DATE
#                                                           from DMRT.DM_811_BASE                                                      
# )
 
# SELECT
# A.ACCNT_TYPE_CODE, a.CRN, a.ACCNT_NUM, a.PREMATURE_CLSRE,a.ACCNT_CLS_DATE, a.ACCNT_MATURITY_DATE,a.ACCNT_OPN_DATE,a.CLS_AT_MATURITY_FLAG,
# a.IS_BULK_DEPST, cast(a.INITIAL_DEPST_AMT as FLOAT) as INITIAL_DEPST_AMT, B.CRN as '811_CRN'
# FROM
# CDR.DIM_TERM_DEPOSIT_ACCOUNT A
# LEFT JOIN CTE1 B
# ON A.CRN  = B.CRN
# WHERE
#   ( A.CURRENT_FLAG='Y')
#    AND
# (    (DATEDIFF(MONTH ,   cast(A.ACCNT_OPN_DATE as DATE), getdate())) = 1 )

In [1]:
import numpy as np
import pandas as pd
import glob
from datetime import datetime,date
pd.set_option('display.float_format', str)
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import os
from xlsx2csv import Xlsx2csv
from io import StringIO
import pandas as pd
import time

In [2]:
## MTD adjustment
folder=r"D:/Portfolio metrics Products Teams/Python implementation/input_data/August2023/"
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import timedelta
today1='2025-03-31'   ## Last 
today = pd.to_datetime(today1).date()
this_month_start=today.replace(day=1)
last_month_today = today - relativedelta(months=1)
last_month_start=last_month_today.replace(day=1)
last_month_last_day = this_month_start - timedelta(days=1)

In [12]:

fd_rd_sweep=pd.read_csv("/Users/kmbl227932/Downloads/monthendreport11.csv")
fd_rd_sweep.drop_duplicates( keep="last", inplace=True)
# importing existing CRNs
df_migration= pd.read_csv('/Users/kmbl227932/Downloads/outward crns.csv')
df_migration.drop_duplicates(subset='crn',keep="last", inplace=True)

#importing FD RD product categories file
# product_fd_rd= pd.read_excel(r'/Users/kmbl227932/Downloads/product.xlsx')



In [16]:
fd_rd_sweep.columns=fd_rd_sweep.columns.str.upper()

In [18]:
def product(schme_code):
    if 'FF' in schme_code:
        return  'SWEEP'
    elif 'RD' in schme_code:
        return 'RD'
    else:
        return 'Standalone'


fd_rd_sweep['Product']= fd_rd_sweep['ACCNT_TYPE_CODE'].apply(product)


In [19]:
fd_rd_sweep_accounts=fd_rd_sweep

In [20]:
fd_rd_sweep_accounts_11=fd_rd_sweep_accounts.merge(df_migration,left_on='CRN', right_on='crn', how='left', indicator=True)


In [21]:
fd_rd_sweep_accounts_11['Pre_migration']=np.where(fd_rd_sweep_accounts_11['811_CRN'].notnull(),'811','Non-811')
fd_rd_sweep_accounts_11

,ACCNT_TYPE_CODE,CRN,ACCNT_NUM,ACCNT_CLS_DATE,ACCNT_MATURITY_DATE,ACCNT_OPN_DATE,INITIAL_DEPST_AMT,811_CRN,Product,crn,outward,_merge,Pre_migration
0,TRDGEN,117891812,1750275946,NaN,2026-03-11,2025-03-11,1000.0,117891812.0,RD,NaN,NaN,left_only,811
1,TFF811,117982167,6784550945,NaN,2025-09-27,2025-03-31,80000.0,117982167.0,SWEEP,NaN,NaN,left_only,811
2,TFF811,117982167,6783385225,2025-03-21,2025-09-09,2025-03-13,5000.0,117982167.0,SWEEP,NaN,NaN,left_only,811
3,TFF811,117982167,6783987636,NaN,2025-09-20,2025-03-24,20000.0,117982167.0,SWEEP,NaN,NaN,left_only,811
4,TFF811,118084815,1582907082,NaN,2025-09-02,2025-03-06,25000.0,118084815.0,SWEEP,NaN,NaN,left_only,811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488543,TFFJIF,194809978,7883425025,NaN,2025-09-10,2025-03-14,45000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811
488544,TFFJIF,90291766,6682561069,NaN,2025-08-28,2025-03-01,50000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811
488545,TFFJIF,90291766,6683465252,NaN,2025-09-11,2025-03-15,10000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811
488546,TFFJIF,186490971,7183667093,2025-04-07,2025-09-15,2025-03-19,350000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811


In [22]:
fd_rd_sweep_accounts_11['Post_migration']=np.where((fd_rd_sweep_accounts_11['Pre_migration']=='811') & (fd_rd_sweep_accounts_11['_merge']=='both'),'Non-811',fd_rd_sweep_accounts_11['Pre_migration'])
fd_rd_sweep_accounts_11

,ACCNT_TYPE_CODE,CRN,ACCNT_NUM,ACCNT_CLS_DATE,ACCNT_MATURITY_DATE,ACCNT_OPN_DATE,INITIAL_DEPST_AMT,811_CRN,Product,crn,outward,_merge,Pre_migration,Post_migration
0,TRDGEN,117891812,1750275946,NaN,2026-03-11,2025-03-11,1000.0,117891812.0,RD,NaN,NaN,left_only,811,811
1,TFF811,117982167,6784550945,NaN,2025-09-27,2025-03-31,80000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
2,TFF811,117982167,6783385225,2025-03-21,2025-09-09,2025-03-13,5000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
3,TFF811,117982167,6783987636,NaN,2025-09-20,2025-03-24,20000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
4,TFF811,118084815,1582907082,NaN,2025-09-02,2025-03-06,25000.0,118084815.0,SWEEP,NaN,NaN,left_only,811,811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488543,TFFJIF,194809978,7883425025,NaN,2025-09-10,2025-03-14,45000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488544,TFFJIF,90291766,6682561069,NaN,2025-08-28,2025-03-01,50000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488545,TFFJIF,90291766,6683465252,NaN,2025-09-11,2025-03-15,10000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488546,TFFJIF,186490971,7183667093,2025-04-07,2025-09-15,2025-03-19,350000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811


In [23]:
fd_rd_sweep_accounts_11['Pre_migration'].value_counts()

Pre_migration
811        478671
Non-811      9877
Name: count, dtype: int64

In [24]:
fd_rd_sweep_accounts_11['Post_migration'].value_counts()

Post_migration
811        476850
Non-811     11698
Name: count, dtype: int64

In [25]:
fd_rd_sweep_accounts_11['Post_migration'].value_counts()

Post_migration
811        476850
Non-811     11698
Name: count, dtype: int64

In [26]:
# fd_rd_sweep_accounts=pd.merge(fd_rd_sweep, product_fd_rd, left_on='ACCNT_TYPE_CODE', right_on='ACCNT_TYPE_CODE', how='left')
fd_rd_sweep_811=fd_rd_sweep_accounts_11.copy()
fd_rd_sweep_811['ACCNT_OPN_DATE'] = pd.to_datetime(fd_rd_sweep_811['ACCNT_OPN_DATE']).dt.date
fd_rd_sweep_811['ACCNT_CLS_DATE'] = pd.to_datetime(fd_rd_sweep_811['ACCNT_CLS_DATE']).dt.date


# Make Account closing date greater than today as NAN 
# fd_rd_sweep_811[fd_rd_sweep_811['ACCNT_CLS_DATE']>pd.to_datetime(today).date()]['ACCNT_CLS_DATE']=np.nan
fd_rd_sweep_811['ACCNT_CLS_DATE']=np.where(fd_rd_sweep_811['ACCNT_CLS_DATE']>pd.to_datetime(today).date(),np.nan,fd_rd_sweep_811['ACCNT_CLS_DATE'])

In [27]:
fd_rd_sweep_811

,ACCNT_TYPE_CODE,CRN,ACCNT_NUM,ACCNT_CLS_DATE,ACCNT_MATURITY_DATE,ACCNT_OPN_DATE,INITIAL_DEPST_AMT,811_CRN,Product,crn,outward,_merge,Pre_migration,Post_migration
0,TRDGEN,117891812,1750275946,NaT,2026-03-11,2025-03-11,1000.0,117891812.0,RD,NaN,NaN,left_only,811,811
1,TFF811,117982167,6784550945,NaT,2025-09-27,2025-03-31,80000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
2,TFF811,117982167,6783385225,2025-03-21,2025-09-09,2025-03-13,5000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
3,TFF811,117982167,6783987636,NaT,2025-09-20,2025-03-24,20000.0,117982167.0,SWEEP,NaN,NaN,left_only,811,811
4,TFF811,118084815,1582907082,NaT,2025-09-02,2025-03-06,25000.0,118084815.0,SWEEP,NaN,NaN,left_only,811,811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488543,TFFJIF,194809978,7883425025,NaT,2025-09-10,2025-03-14,45000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488544,TFFJIF,90291766,6682561069,NaT,2025-08-28,2025-03-01,50000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488545,TFFJIF,90291766,6683465252,NaT,2025-09-11,2025-03-15,10000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811
488546,TFFJIF,186490971,7183667093,NaN,2025-09-15,2025-03-19,350000.0,NaN,SWEEP,NaN,NaN,left_only,Non-811,Non-811


In [29]:
#811 Pre-Migration Cx

fd_rd_sweep_811_group=fd_rd_sweep_811[(fd_rd_sweep_811['ACCNT_CLS_DATE'].isnull()) & (fd_rd_sweep_811['Pre_migration']=='811')].groupby(['ACCNT_OPN_DATE','Product'], dropna=False).agg({'ACCNT_NUM':'count', 'INITIAL_DEPST_AMT': 'sum'}).reset_index().rename(columns={'ACCNT_NUM':'Count','INITIAL_DEPST_AMT':'Amount'})
fd_rd_sweep_811_count_amount=fd_rd_sweep_811_group.pivot_table(index=['ACCNT_OPN_DATE'], columns='Product', values=['Count','Amount']).reset_index()
fd_rd_sweep_811_count_amount.columns = [' '.join(col).strip() for col in fd_rd_sweep_811_count_amount.columns.values]
amount=['Amount RD', 'Amount SWEEP', 'Amount Standalone']
fd_rd_sweep_811_count_amount[amount]=(fd_rd_sweep_811_count_amount[amount].astype(float)/10000000).round(2)
fd_rd_sweep_811_count_amount=fd_rd_sweep_811_count_amount[['ACCNT_OPN_DATE','Count Standalone', 'Amount Standalone','Count RD', 'Amount RD','Count SWEEP', 'Amount SWEEP']]
fd_rd_sweep_811_count_amount.rename(columns={'Count Standalone':'Count FD','Amount Standalone':'Amount FD(Cr)','Amount RD':'Amount RD(Cr)','Amount SWEEP':'Amount SWEEP(Cr)'}, inplace=True)


In [30]:
#811 Pre-Migration Cx

fd_rd_sweep_811_count_amount.loc['Sum']=round(fd_rd_sweep_811_count_amount.drop('ACCNT_OPN_DATE',axis=1).sum(),2)
fd_rd_sweep_811_count_amount.loc['Sum','ACCNT_OPN_DATE']="Total"
fd_rd_sweep_811_count_amount


,ACCNT_OPN_DATE,Count FD,Amount FD(Cr),Count RD,Amount RD(Cr),Count SWEEP,Amount SWEEP(Cr)
0,2025-03-01,2232.0,16.43,1124.0,0.44,4097.0,19.61
1,2025-03-02,1574.0,10.23,821.0,0.33,2597.0,9.64
2,2025-03-03,1742.0,13.67,822.0,0.29,3346.0,14.89
3,2025-03-04,1685.0,12.86,762.0,0.26,3247.0,15.47
4,2025-03-05,1837.0,14.4,803.0,0.26,3605.0,17.18
5,2025-03-06,1717.0,13.72,805.0,0.24,3505.0,15.4
6,2025-03-07,1985.0,13.62,752.0,0.27,4052.0,15.93
7,2025-03-08,1523.0,9.9,729.0,0.2,3326.0,13.45
8,2025-03-09,1261.0,8.37,641.0,0.2,2610.0,9.5
9,2025-03-10,1856.0,15.04,824.0,0.24,3990.0,16.47


In [31]:
fd_rd_sweep_811_group

,ACCNT_OPN_DATE,Product,Count,Amount
0,2025-03-01,RD,1124,4408700.0
1,2025-03-01,SWEEP,4097,196148000.0
2,2025-03-01,Standalone,2232,164319190.0
3,2025-03-02,RD,821,3349400.0
4,2025-03-02,SWEEP,2597,96449000.0
...,...,...,...,...
88,2025-03-30,SWEEP,8250,281391000.0
89,2025-03-30,Standalone,2100,144165527.0
90,2025-03-31,RD,1123,4258700.0
91,2025-03-31,SWEEP,12491,442370010.0


In [32]:
# Non-811 Pre-Migration Cx

fd_rd_sweep_non_811_group=fd_rd_sweep_811[(fd_rd_sweep_811['ACCNT_CLS_DATE'].isnull()) & (fd_rd_sweep_811['Pre_migration']=='Non-811')].groupby(['ACCNT_OPN_DATE','Product'], dropna=False).agg({'ACCNT_NUM':'count', 'INITIAL_DEPST_AMT': 'sum'}).reset_index().rename(columns={'ACCNT_NUM':'Count','INITIAL_DEPST_AMT':'Amount'})
fd_rd_sweep_non_811_count_amount=fd_rd_sweep_non_811_group.pivot_table(index=['ACCNT_OPN_DATE'], columns='Product', values=['Count','Amount']).reset_index()
fd_rd_sweep_non_811_count_amount.columns=[' '.join(col).strip() for col in fd_rd_sweep_non_811_count_amount.columns.values]
amount=['Amount RD', 'Amount SWEEP', 'Amount Standalone']
fd_rd_sweep_non_811_count_amount[amount]=(fd_rd_sweep_non_811_count_amount[amount].astype(float)/10000000).round(2)
fd_rd_sweep_non_811_count_amount=fd_rd_sweep_non_811_count_amount[['ACCNT_OPN_DATE','Count Standalone', 'Amount Standalone','Count RD', 'Amount RD','Count SWEEP', 'Amount SWEEP']]
fd_rd_sweep_non_811_count_amount.rename(columns={'Count Standalone':'Count FD','Amount Standalone':'Amount FD(Cr)','Amount RD':'Amount RD(Cr)','Amount SWEEP':'Amount SWEEP(Cr)'}, inplace=True)

fd_rd_sweep_non_811_count_amount.loc['Sum']=round(fd_rd_sweep_non_811_count_amount.drop('ACCNT_OPN_DATE',axis=1).sum(),2)
fd_rd_sweep_non_811_count_amount.loc['Sum','ACCNT_OPN_DATE']="Total"

fd_rd_sweep_non_811_count_amount

,ACCNT_OPN_DATE,Count FD,Amount FD(Cr),Count RD,Amount RD(Cr),Count SWEEP,Amount SWEEP(Cr)
0,2025-03-01,18.0,0.21,10.0,0.01,97.0,0.87
1,2025-03-02,8.0,0.07,2.0,0.01,70.0,0.68
2,2025-03-03,14.0,0.23,4.0,0.0,78.0,0.55
3,2025-03-04,6.0,0.08,5.0,0.0,93.0,1.34
4,2025-03-05,11.0,0.19,2.0,0.0,72.0,0.39
5,2025-03-06,9.0,0.07,2.0,0.0,75.0,0.78
6,2025-03-07,9.0,0.08,2.0,0.0,89.0,0.72
7,2025-03-08,10.0,0.03,1.0,0.0,58.0,0.45
8,2025-03-09,8.0,0.13,5.0,0.0,53.0,0.32
9,2025-03-10,10.0,0.17,2.0,0.0,97.0,0.77


In [33]:
#811 Post-Migration Cx

fd_rd_sweep_811_post_group=fd_rd_sweep_811[ (fd_rd_sweep_811['ACCNT_CLS_DATE'].isnull()) & (fd_rd_sweep_811['Post_migration']=='811')].groupby(['ACCNT_OPN_DATE','Product'], dropna=False).agg({'ACCNT_NUM':'count', 'INITIAL_DEPST_AMT': 'sum'}).reset_index().rename(columns={'ACCNT_NUM':'Count','INITIAL_DEPST_AMT':'Amount'})
fd_rd_sweep_811_count_post_amount=fd_rd_sweep_811_post_group.pivot_table(index=['ACCNT_OPN_DATE'], columns='Product', values=['Count','Amount']).reset_index()
fd_rd_sweep_811_count_post_amount.columns = [' '.join(col).strip() for col in fd_rd_sweep_811_count_post_amount.columns.values]
amount=['Amount RD', 'Amount SWEEP', 'Amount Standalone']
fd_rd_sweep_811_count_post_amount[amount]=(fd_rd_sweep_811_count_post_amount[amount].astype(float)/10000000).round(2)
fd_rd_sweep_811_count_post_amount=fd_rd_sweep_811_count_post_amount[['ACCNT_OPN_DATE','Count Standalone', 'Amount Standalone','Count RD', 'Amount RD','Count SWEEP', 'Amount SWEEP']]
fd_rd_sweep_811_count_post_amount.rename(columns={'Count Standalone':'Count FD','Amount Standalone':'Amount FD(Cr)','Amount RD':'Amount RD(Cr)','Amount SWEEP':'Amount SWEEP(Cr)'}, inplace=True)
fd_rd_sweep_811_count_post_amount.loc['Sum']=round(fd_rd_sweep_811_count_post_amount.drop('ACCNT_OPN_DATE',axis=1).sum(),2)
fd_rd_sweep_811_count_post_amount.loc['Sum','ACCNT_OPN_DATE']="Total"
fd_rd_sweep_811_count_post_amount

,ACCNT_OPN_DATE,Count FD,Amount FD(Cr),Count RD,Amount RD(Cr),Count SWEEP,Amount SWEEP(Cr)
0,2025-03-01,2222.0,16.26,1120.0,0.44,4076.0,19.49
1,2025-03-02,1568.0,10.12,818.0,0.33,2583.0,9.48
2,2025-03-03,1737.0,13.61,818.0,0.28,3333.0,14.76
3,2025-03-04,1682.0,12.77,761.0,0.26,3238.0,15.38
4,2025-03-05,1833.0,14.37,802.0,0.26,3591.0,17.12
5,2025-03-06,1711.0,13.7,804.0,0.23,3486.0,15.26
6,2025-03-07,1983.0,13.56,750.0,0.26,4037.0,15.85
7,2025-03-08,1519.0,9.88,726.0,0.2,3311.0,13.3
8,2025-03-09,1259.0,8.33,640.0,0.2,2596.0,9.44
9,2025-03-10,1853.0,15.02,822.0,0.23,3972.0,16.33


In [34]:
# Non-811 Post-Migration Cx

fd_rd_sweep_non_811_post_group=fd_rd_sweep_811[ (fd_rd_sweep_811['ACCNT_CLS_DATE'].isnull()) & (fd_rd_sweep_811['Post_migration']=='Non-811')].groupby(['ACCNT_OPN_DATE','Product'], dropna=False).agg({'ACCNT_NUM':'count', 'INITIAL_DEPST_AMT': 'sum'}).reset_index().rename(columns={'ACCNT_NUM':'Count','INITIAL_DEPST_AMT':'Amount'})
fd_rd_sweep_non_811_count_post_amount=fd_rd_sweep_non_811_post_group.pivot_table(index=['ACCNT_OPN_DATE'], columns='Product', values=['Count','Amount']).reset_index()
fd_rd_sweep_non_811_count_post_amount.columns=[' '.join(col).strip() for col in fd_rd_sweep_non_811_count_post_amount.columns.values]
amount=['Amount RD', 'Amount SWEEP', 'Amount Standalone']
fd_rd_sweep_non_811_count_post_amount[amount]=(fd_rd_sweep_non_811_count_post_amount[amount].astype(float)/10000000).round(2)
fd_rd_sweep_non_811_count_post_amount=fd_rd_sweep_non_811_count_post_amount[['ACCNT_OPN_DATE','Count Standalone', 'Amount Standalone','Count RD', 'Amount RD','Count SWEEP', 'Amount SWEEP']]
fd_rd_sweep_non_811_count_post_amount.rename(columns={'Count Standalone':'Count FD','Amount Standalone':'Amount FD(Cr)','Amount RD':'Amount RD(Cr)','Amount SWEEP':'Amount SWEEP(Cr)'}, inplace=True)

fd_rd_sweep_non_811_count_post_amount.loc['Sum']=round(fd_rd_sweep_non_811_count_post_amount.drop('ACCNT_OPN_DATE',axis=1).sum(),2)
fd_rd_sweep_non_811_count_post_amount.loc['Sum','ACCNT_OPN_DATE']="Total"
fd_rd_sweep_non_811_count_post_amount

,ACCNT_OPN_DATE,Count FD,Amount FD(Cr),Count RD,Amount RD(Cr),Count SWEEP,Amount SWEEP(Cr)
0,2025-03-01,28.0,0.39,14.0,0.01,118.0,0.99
1,2025-03-02,14.0,0.18,5.0,0.01,84.0,0.84
2,2025-03-03,19.0,0.29,8.0,0.01,91.0,0.68
3,2025-03-04,9.0,0.17,6.0,0.0,102.0,1.43
4,2025-03-05,15.0,0.21,3.0,0.0,86.0,0.45
5,2025-03-06,15.0,0.1,3.0,0.0,94.0,0.91
6,2025-03-07,11.0,0.15,4.0,0.0,104.0,0.8
7,2025-03-08,14.0,0.05,4.0,0.0,73.0,0.6
8,2025-03-09,10.0,0.17,6.0,0.01,67.0,0.37
9,2025-03-10,13.0,0.2,4.0,0.01,115.0,0.91


In [35]:
writer = pd.ExcelWriter('/Users/kmbl227932/Desktop/Anashwara/projects/FDRD/FD_RD_mar_2025.xlsx', engine='xlsxwriter')

fd_rd_sweep_811_count_amount.to_excel(writer, sheet_name='Pre-migration-811', index=False)

fd_rd_sweep_non_811_count_amount.to_excel(writer, sheet_name='Pre-migration-non-811', index=False)

fd_rd_sweep_811_count_post_amount.to_excel(writer, sheet_name='Post-migration-811', index=False)

fd_rd_sweep_non_811_count_post_amount.to_excel(writer, sheet_name='Post-migration-Non-811', index=False)

#df3.to_excel(writer, sheet_name='Sheetc')

writer.close()